# Tequile Demo (BROKEN)

Demo from Jakob Kottman for G1 VQE optimization using H2

In [1]:
import tequila as tq
import numpy

### Part 1: Initialize a Molecule (H2) and Optimize the Orbitals

In [16]:
geometry = "H 0.0 0.0 0.0\nH 0.0 0.0 1.0"
mol = tq.Molecule(geometry=geometry, basis_set="STO-3G") # STO-3G basically means one orbital per H-Atom

# exact energy
fci = mol.compute_energy("fci")
print(f"Exact energy: {fci}")

# represent with atomic orbitals (easier to create the guess further down)
mol = mol.use_native_orbitals()

Exact energy: -1.101150330232619


In [17]:
U = mol.make_ansatz(name="SPA", edges=[(0,1)])

guess = numpy.asarray([[1.,1.],[1.,-1.]])

opt = tq.chemistry.optimize_orbitals(circuit=U, molecule=mol, initial_guess=guess.T, silent=True)
print(f"Optimized energy: {opt.energy}")

# molecule with optimized orbitals for the SPA wavefunction
mol = opt.molecule

Optimized energy: -1.101150330132665


/home/liamj/chemical-graph/chemenv/lib/python3.12/site-packages/pyscf/gto/mole.py:1293: UserWarning: Function mol.dumps drops attribute energy_nuc because it is not JSON-serializable
  warnings.warn(msg)


### Part2: G1 Method without Hardware-Friendly Optimization

In [18]:
# circuit
U = mol.make_ansatz(name="SPA", edges=[(0,1)])

# hamiltonian
H = mol.make_hamiltonian()

# expectation value
E = tq.ExpectationValue(H=H, U=U)

# VQE
result = tq.minimize(E, silent=True)
print("exact energy: {:+2.7f} ".format(fci))
print("spa energy  : {:+2.7f} ".format(result.energy))

exact energy: -1.1011503 
spa energy  : -1.1011503 


### Part3: G1 Method with Hardware-Friendly Optimization

We are gonna employ the Hardcore-Boson approximation (treat electron-pairs as quasi-particles). G1 (or SPA) circuits are producing wavefunction within that approximation. 

Net effect:
- only half the qubits needed
- less measurements (Hamiltonian has naturally less terms)

Down side:
- needs a bit more tweaking in the code 

In [22]:
# get the qubit Hamiltonian
H = mol.make_hardcore_boson_hamiltonian()
# create the SPA ansatz in the HCB representation
U = mol.make_ansatz(name="HCB-SPA", edges=[(0,1)])

# the following is due to the warning displayed below
# HCB-SPA is defined on the even qubits: 0,2,4,...
# in order to be consistent with the Hamiltonian above we map it to 0,1,2....
# sorry about this, work in progress to make things more consistent
# U = U.map_qubits({2*k:k for k in H.qubits})

# create an expecationvalue 
E = tq.ExpectationValue(H=H, U=U)
print(H)
print(U)
print(E)

# get optimal angles
result = tq.minimize(E, silent=True)
opt_angles = result.variables
print()

# diagonalize hamiltonian
v = numpy.linalg.eigvalsh(H.to_matrix())

print("exact energy: {:+2.5f} ".format(fci))
print("spa energy  : {:+2.5f} ".format(result.energy))
print("exact energy in hcb approximation: ", v[0])

# compute wavefunctions
wfn = tq.simulate(U, opt_angles)
print("VQE Wavefunction")
print(wfn)

-0.0077+0.2743Z(0)+0.0984X(0)X(2)+0.0984Y(0)Y(2)-0.2607Z(2)+0.5233Z(0)Z(2)
circuit: 
X(target=(0,))
Ry(target=(2,), parameter=((0, 1), 'D', None))
X(target=(0,), control=(2,))

Objective with 1 unique expectation values
total measurements = 6
variables          = [((0, 1), 'D', None)]
types              = not compiled



exact energy: -1.10115 
spa energy  : -1.10115 
exact energy in hcb approximation:  -1.101150330232618
VQE Wavefunction
+0.9845 |100> -0.1753 |001> 


In [23]:
print(opt_angles)

((0, 1), 'D', None) : -0.3524189750081696



### Part4: G1 Method with Hardware-Friendly Optimization and Measurement Optimization

We are gonna employ measurement optimization techniques. In this case it's gonna reduce the measurements of our Hamiltonian from 6 to 3.

- before: 6 individual Pauli-Strings
- after: 3 Expectationvalues, each with an all-Z Pauli

Down side:
- circuits get larger
- in this case: not that bad after manual optimization

In [24]:
# create an expectation vaue
# 6 measurements since there are 6 Pauli-Strings in the Hamiltonian above
E = tq.ExpectationValue(H=H, U=U)
print(E)

# evaluate with shots
shots=1000
energies=[tq.simulate(E, opt_angles, samples=shots) for _ in range(100)]
# min, max, average
print(min(energies))
print(max(energies))
print(sum(energies)/100)

Objective with 1 unique expectation values
total measurements = 6
variables          = [((0, 1), 'D', None)]
types              = not compiled
-1.1129699601450658
-1.0886820782262001
-1.10138733470916


Without measurement optimization

In [10]:
E2 = tq.compile(E, backend="cirq")
for e in E2.get_expectationvalues():
    print("this is the circuit")
    print(e.U.circuit)
    print("this is the Hamiltonian")
    print(e.H)
    print("")
    print("")

this is the circuit
0: ───X───────────────────────────────────────────────────X───
                                                          │
1: ───Y^(0.318309886183791*f((((0, 1), 'D', None),))_0)───@───
this is the Hamiltonian
(-0.0077+0.2743Z(0)+0.0984X(0)X(2)+0.0984Y(0)Y(2)-0.2607Z(2)+0.5233Z(0)Z(2),)




With measurement optimization

In [27]:
# now with measurement optimization
options = {"method":"si", "condition": "qwc"}
E = tq.ExpectationValue(H=H, U=U, optimize_measurements=options)
print(E)

# evaluate with shots
shots=1000
energies=[tq.simulate(E, opt_angles, samples=shots) for _ in range(100)]
# min, max, average
print(min(energies))
print(max(energies))
print(sum(energies)/100)

Objective with 3 unique expectation values
total measurements = 3
variables          = [((0, 1), 'D', None)]
types              = not compiled
-1.050486685900058
-1.0163429932488004
-1.0326256145796773


In [30]:
# here you can get the individual circuits and the measurement instructions
# the total expectation value is then everything accumulated
# you see that the measurement optimization is adding stuff to the circuit
# in this case it's however just a single qubit rotation
# the automatic procedure is however not perfect, as you can see :-) 
for e in E.get_expectationvalues():
    print("this is the circuit")
    print(e.U)
    print("this is the Hamiltonian")
    print(e.H)

this is the circuit
circuit: 
X(target=(0,))
Ry(target=(2,), parameter=((0, 1), 'D', None))
X(target=(0,), control=(2,))

this is the Hamiltonian
(+0.5233Z(0)Z(2)+0.2743Z(0)-0.2607Z(2)-0.0077,)
this is the circuit
circuit: 
X(target=(0,))
Ry(target=(2,), parameter=((0, 1), 'D', None))
X(target=(0,), control=(2,))
H(target=(2,))
H(target=(1,))
H(target=(0,))
Phase(target=(2,), parameter=1.5707963267948966)
Phase(target=(2,), parameter=1.5707963267948966)
Phase(target=(1,), parameter=1.5707963267948966)
Phase(target=(1,), parameter=1.5707963267948966)
Phase(target=(0,), parameter=1.5707963267948966)
Phase(target=(0,), parameter=1.5707963267948966)
Phase(target=(2,), parameter=1.5707963267948966)
Phase(target=(1,), parameter=1.5707963267948966)
Phase(target=(0,), parameter=1.5707963267948966)
Phase(target=(2,), parameter=1.5707963267948966)
Phase(target=(1,), parameter=1.5707963267948966)
Phase(target=(0,), parameter=1.5707963267948966)
H(target=(1,))

this is the Hamiltonian
(+0.0984Z(0)

In [31]:
# here is the same, but with the circuits in cirq objects
# you can also replace backend="qiskit" 
E2 = tq.compile(E, backend="cirq")
for e in E2.get_expectationvalues():
    print("this is the circuit")
    print(e.U.circuit)
    print("this is the Hamiltonian")
    print(e.H)
    print("")
    print("")

this is the circuit
0: ───X───────────────────────────────────────────────────X───
                                                          │
1: ───Y^(0.318309886183791*f((((0, 1), 'D', None),))_0)───@───
this is the Hamiltonian
(-0.0077+0.5233Z(0)Z(2)+0.2743Z(0)-0.2607Z(2),)


this is the circuit
                                                          ┌──┐
0: ───X────────────────────────────────────────────────────X─────H───S───S───S───S───
                                                           │
1: ───H────────────────────────────────────────────────────┼S────S───S───S───H───────
                                                           │
2: ───Y^(0.318309886183791*f((((0, 1), 'D', None),))_0)────@─────H───S───S───S───S───
                                                          └──┘
this is the Hamiltonian
(+0.0984Z(0)Z(2),)


this is the circuit
                                                          ┌──┐
0: ───X────────────────────────────────────────────────────X─────S